In [1]:
import sys

!{sys.executable} -m pip install pdftotext

You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# info: https://www.uwv.nl/overuwv/pers/documenten/2020/gegevens-ontvangers-now-1-0-regeling.aspx
# pdf: https://www.simulatienow.nl/Register-NOW-100720.pdf

import requests
from pathlib import Path

reports = [
    'Register-NOW-100720.pdf',
]

outputfolder = Path('./')

for report in reports:
    url = f'https://www.simulatienow.nl/{report}'
    
    outputfile = outputfolder / f'now-register-{report.split("-")[-1]}'
    
    if not outputfile.exists():
        r = requests.get(url, allow_redirects=True)
        open(outputfile, 'wb').write(r.content)

In [3]:
# I'm using poppler pdfinfo because it was available and it performs quite fast
# compared to alternatives using pdfminer library. - Thomas

# source: https://gist.github.com/godber/7692812#file-pdfinfo-py

import subprocess
import os.path as osp


def pdfinfo(infile):
    """
    Wraps command line utility pdfinfo to extract the PDF meta information.
    Returns metainfo in a dictionary.
    sudo apt-get install poppler-utils
    This function parses the text output that looks like this:
        Title:          PUBLIC MEETING AGENDA
        Author:         Customer Support
        Creator:        Microsoft Word 2010
        Producer:       Microsoft Word 2010
        CreationDate:   Thu Dec 20 14:44:56 2012
        ModDate:        Thu Dec 20 14:44:56 2012
        Tagged:         yes
        Pages:          2
        Encrypted:      no
        Page size:      612 x 792 pts (letter)
        File size:      104739 bytes
        Optimized:      no
        PDF version:    1.5
    """

    cmd = '/usr/bin/pdfinfo'
    if not osp.exists(cmd):
        raise RuntimeError('System command not found: %s' % cmd)

    if not osp.exists(infile):
        raise RuntimeError('Provided input file not found: %s' % infile)

    def _extract(row):
        """Extracts the right hand value from a : delimited row"""
        return row.split(':', 1)[1].strip()

    output = {}

    labels = ['Title', 'Author', 'Creator', 'Producer', 'CreationDate',
              'ModDate', 'Tagged', 'Pages', 'Encrypted', 'Page size',
              'File size', 'Optimized', 'PDF version']

    cmd_output = subprocess.check_output([cmd, infile])
    for line in map(str, cmd_output.splitlines()):
        for label in labels:
            if label in line:
                output[label] = _extract(line)

    return output

In [4]:
# some more utility functions

import pdftotext
import re
    
def read_dutch_float(num):
    return float(num.replace('.', '').replace(',', '.'))
    
def extract_page(page):
    column_names = None
    data = []
    
    column_phase = True
    
    for row in page.splitlines():
        fields = re.split(r'\s{2,}', row.strip())
        
        
        if column_phase and column_names is None:
            if len(fields) == 2:
                column_names = ' '.join([x.strip() for x in fields]).split(' ')  # reassemble and split
            else:
                column_names = fields
            #print(column_names)
        elif column_phase and len(fields) < 3:
            column_names[2] += ' ' + fields[0]
            #print(column_names)
            column_phase = False
        else:
            if len(fields) == 1:
                try:
                    int(fields[0])  # check if this is a pagenum
                    continue # if it is, just skip to the next
                except ValueError as e:  # unfortunately it is not, most likely it is now part of the company location or company name
                    print('NOT PAGENUM', fields)
                    print(f'--{row}--')
                    
                    if row.lstrip() != row:  # check if it has a bunch of spaces in front
                        # it's a location
                        print('Assuming location')
                        data[len(data) -1][1] += f'{fields[0]}'
                    else:
                        # it's part of the companies name
                        print('Assuming company name')
                        data[len(data) -1][0] += f' {fields[0]}'
                    continue  # on to the next row
            elif len(fields) == 2:
                # placename is probably not extracted due to not having two whitespace chars between
                # the columns, let's split them anyways
                
                fields_new = [
                    ' '.join(fields[0].split(' ')[:-1]),
                    ' '.join(fields[0].split(' ')[-1:]),
                    fields[1]
                ]
                fields = fields_new
            elif len(fields) != 3:
                print('ERROR', fields) # something's not right
                continue
                
            fields[2] = read_dutch_float(fields[2])
            data.append(fields)
    
    return column_names, data

def pdf_find_page_for_text(file, text):
    with open(file, "rb") as fh:
        pdf = pdftotext.PDF(fh)

    counter = 1
    for page in pdf:
        if text.lower() in page.lower():
            return counter
        counter += 1

In [5]:
import dateutil.parser
from datetime import datetime
import pytz
import pandas as pd


now_registries = outputfolder.glob('now-register-*.pdf')
by_date = {}

for registry in now_registries:
    meta = pdfinfo(registry)
    
    moddate = meta['ModDate'].strip("'").strip('"').strip()  # cleanup the string a bit
    
    moddate = dateutil.parser.parse(moddate)
    #moddate = int(f'{moddate.year}{moddate.month:02}{moddate.day:02}')
    
    by_date[moddate] = registry
    

output = []
    

    
# walk trough reports old to new
for key in sorted(by_date.keys()):
    registry_file = by_date[key]
    table_start = pdf_find_page_for_text(registry_file, 'VOORSCHOTBEDRAG')
    
    #print(table_start)
    
    with open(registry_file, "rb") as fh:
        pdf = pdftotext.PDF(fh)
    
    
    pagenum = table_start
    totalpages = len(pdf) - 2  # skip last few pages, they are colofon so no table data
    for page in range(pagenum, totalpages):
        print(f'Page {page}/{totalpages - 1}')
        columns, data = extract_page(pdf[page])
        output = output + data  # concat two lists of lists
        
    df = pd.DataFrame(output, columns=columns)
    outputname = f'now-registry-{key.year}{key.month:02}{key.day:02}.csv'
    print(f'Writing to {outputname}')
    df.to_csv(outputname)


Page 2/2045
Page 3/2045
Page 4/2045
Page 5/2045
Page 6/2045
Page 7/2045
Page 8/2045
Page 9/2045
Page 10/2045
Page 11/2045
Page 12/2045
Page 13/2045
Page 14/2045
Page 15/2045
Page 16/2045
Page 17/2045
Page 18/2045
Page 19/2045
Page 20/2045
Page 21/2045
Page 22/2045
Page 23/2045
Page 24/2045
Page 25/2045
Page 26/2045
Page 27/2045
Page 28/2045
Page 29/2045
Page 30/2045
Page 31/2045
Page 32/2045
Page 33/2045
Page 34/2045
Page 35/2045
Page 36/2045
Page 37/2045
NOT PAGENUM ['ALBRANDSWAARD']
--                                                   ALBRANDSWAARD--
Assuming location
Page 38/2045
Page 39/2045
Page 40/2045
Page 41/2045
Page 42/2045
Page 43/2045
Page 44/2045
Page 45/2045
Page 46/2045
Page 47/2045
Page 48/2045
Page 49/2045
Page 50/2045
Page 51/2045
Page 52/2045
Page 53/2045
Page 54/2045
Page 55/2045
Page 56/2045
Page 57/2045
Page 58/2045
Page 59/2045
NOT PAGENUM ['ALBRANDSWAARD']
--                                                 ALBRANDSWAARD--
Assuming location
Page 60/2045
Page 61/2

Page 544/2045
Page 545/2045
Page 546/2045
Page 547/2045
Page 548/2045
Page 549/2045
Page 550/2045
Page 551/2045
Page 552/2045
Page 553/2045
Page 554/2045
Page 555/2045
Page 556/2045
Page 557/2045
Page 558/2045
Page 559/2045
Page 560/2045
Page 561/2045
Page 562/2045
Page 563/2045
Page 564/2045
Page 565/2045
Page 566/2045
Page 567/2045
Page 568/2045
Page 569/2045
Page 570/2045
Page 571/2045
Page 572/2045
Page 573/2045
Page 574/2045
Page 575/2045
Page 576/2045
Page 577/2045
Page 578/2045
Page 579/2045
Page 580/2045
Page 581/2045
Page 582/2045
Page 583/2045
Page 584/2045
Page 585/2045
Page 586/2045
Page 587/2045
Page 588/2045
Page 589/2045
Page 590/2045
Page 591/2045
Page 592/2045
Page 593/2045
Page 594/2045
Page 595/2045
Page 596/2045
Page 597/2045
Page 598/2045
Page 599/2045
Page 600/2045
Page 601/2045
Page 602/2045
Page 603/2045
Page 604/2045
Page 605/2045
Page 606/2045
Page 607/2045
Page 608/2045
Page 609/2045
Page 610/2045
Page 611/2045
Page 612/2045
Page 613/2045
Page 614/2045
Page 6

Page 1073/2045
Page 1074/2045
Page 1075/2045
Page 1076/2045
Page 1077/2045
Page 1078/2045
Page 1079/2045
Page 1080/2045
Page 1081/2045
Page 1082/2045
Page 1083/2045
Page 1084/2045
Page 1085/2045
Page 1086/2045
Page 1087/2045
Page 1088/2045
Page 1089/2045
Page 1090/2045
Page 1091/2045
Page 1092/2045
Page 1093/2045
Page 1094/2045
Page 1095/2045
Page 1096/2045
Page 1097/2045
Page 1098/2045
Page 1099/2045
Page 1100/2045
Page 1101/2045
Page 1102/2045
Page 1103/2045
Page 1104/2045
Page 1105/2045
Page 1106/2045
Page 1107/2045
Page 1108/2045
Page 1109/2045
Page 1110/2045
Page 1111/2045
Page 1112/2045
Page 1113/2045
NOT PAGENUM ['ALBRANDSWAARD']
--                                               ALBRANDSWAARD--
Assuming location
NOT PAGENUM ['ALBRANDSWAARD']
--                                               ALBRANDSWAARD--
Assuming location
NOT PAGENUM ['ALBRANDSWAARD']
--                                               ALBRANDSWAARD--
Assuming location
NOT PAGENUM ['ALBRANDSWAARD']
--              

Assuming company name
Page 1493/2045
NOT PAGENUM ['UITERLIJKE VERZORGING']
--UITERLIJKE VERZORGING--
Assuming company name
Page 1494/2045
NOT PAGENUM ['ACTIVITEITENBEGELEIDI']
--ACTIVITEITENBEGELEIDI--
Assuming company name
Page 1495/2045
Page 1496/2045
Page 1497/2045
Page 1498/2045
Page 1499/2045
Page 1500/2045
Page 1501/2045
Page 1502/2045
Page 1503/2045
NOT PAGENUM ['ROTTERDAM']
--ROTTERDAM--
Assuming company name
NOT PAGENUM ['NEDERLAND']
--NEDERLAND--
Assuming company name
Page 1504/2045
Page 1505/2045
Page 1506/2045
Page 1507/2045
NOT PAGENUM ['MIERDEN)']
--MIERDEN)--
Assuming company name
Page 1508/2045
NOT PAGENUM ['SPECIALISES FRANC']
--SPECIALISES FRANC--
Assuming company name
Page 1509/2045
Page 1510/2045
NOT PAGENUM ['GEMEENTE CAST']
--GEMEENTE CAST--
Assuming company name
Page 1511/2045
Page 1512/2045
Page 1513/2045
Page 1514/2045
Page 1515/2045
Page 1516/2045
Page 1517/2045
Page 1518/2045
Page 1519/2045
Page 1520/2045
Page 1521/2045
Page 1522/2045
Page 1523/2045
Page 1524

Page 1938/2045
Page 1939/2045
Page 1940/2045
Page 1941/2045
Page 1942/2045
Page 1943/2045
Page 1944/2045
Page 1945/2045
Page 1946/2045
Page 1947/2045
Page 1948/2045
Page 1949/2045
Page 1950/2045
Page 1951/2045
Page 1952/2045
Page 1953/2045
Page 1954/2045
Page 1955/2045
Page 1956/2045
Page 1957/2045
Page 1958/2045
Page 1959/2045
Page 1960/2045
Page 1961/2045
Page 1962/2045
Page 1963/2045
Page 1964/2045
Page 1965/2045
Page 1966/2045
Page 1967/2045
Page 1968/2045
Page 1969/2045
Page 1970/2045
Page 1971/2045
Page 1972/2045
Page 1973/2045
Page 1974/2045
Page 1975/2045
Page 1976/2045
Page 1977/2045
Page 1978/2045
Page 1979/2045
Page 1980/2045
Page 1981/2045
Page 1982/2045
Page 1983/2045
Page 1984/2045
Page 1985/2045
Page 1986/2045
Page 1987/2045
Page 1988/2045
Page 1989/2045
Page 1990/2045
Page 1991/2045
Page 1992/2045
Page 1993/2045
Page 1994/2045
Page 1995/2045
Page 1996/2045
Page 1997/2045
Page 1998/2045
Page 1999/2045
Page 2000/2045
Page 2001/2045
Page 2002/2045
Page 2003/2045
Page 2004/